In [8]:
import torch
import numpy as np
import time
import torch.nn as nn
from torch.nn import functional as F
# torch.cuda.is_available()
# torch.zeros(1).cuda()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 3000
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
# print(len(text))
# print(text[:1000])
chars = sorted(set(text))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [3]:
string_to_int = {ch: i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: [int_to_string[i] for i in l]

# encoded_text = encode('chitra')
# decoded_text = decode(encoded_text)
# print(encoded_text)
# print(decoded_text)

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[51, 39, 52,  6,  1, 58, 46, 43],
        [39, 61, 42,  2,  0,  0, 28, 13],
        [59, 56, 42, 43, 52,  1, 50, 53],
        [47, 50, 58,  1, 42, 47, 43,  6]], device='cuda:0')
targets:
tensor([[39, 52,  6,  1, 58, 46, 43, 63],
        [61, 42,  2,  0,  0, 28, 13, 33],
        [56, 42, 43, 52,  1, 50, 53, 60],
        [50, 58,  1, 42, 47, 43,  6,  1]], device='cuda:0')


In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=100)[0].tolist())
print(generated_chars)

['\n', 'Z', 'R', 'J', ',', 'x', 'Y', '&', 'B', 'H', 'p', ' ', '\n', '.', 'H', 'Y', 'i', 'E', 'G', 'R', '$', 'f', 'u', 'i', 'f', 'B', 'R', 'u', 'J', 'B', 'Y', '.', 'b', 'K', 'R', 'r', 'Q', 'l', '$', '$', '\n', 'h', 'y', 'U', 'X', 's', 'g', 'n', 'V', 'J', ';', 'b', 'x', 'w', 'S', 'p', 'E', ',', '-', 'Z', 'o', 'j', '?', 'I', '!', 'O', 'J', 'r', 'k', 'u', 'A', 'K', 'z', 'q', 'M', 'Q', 'f', 'v', 'q', 'g', '-', ',', 'G', ' ', 'U', 'A', 'N', 'Y', 'w', 'j', 'i', 'n', 'F', '-', 'S', 's', '&', 'w', '-', 'R', '-']


In [9]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.478, val loss: 4.495
step: 250, train loss: 4.452, val loss: 4.436
step: 500, train loss: 4.387, val loss: 4.388
step: 750, train loss: 4.332, val loss: 4.333
step: 1000, train loss: 4.306, val loss: 4.294
step: 1250, train loss: 4.241, val loss: 4.235
step: 1500, train loss: 4.209, val loss: 4.200
step: 1750, train loss: 4.138, val loss: 4.149
step: 2000, train loss: 4.102, val loss: 4.105
step: 2250, train loss: 4.055, val loss: 4.071
step: 2500, train loss: 4.020, val loss: 4.018
step: 2750, train loss: 3.994, val loss: 3.965
4.010514736175537


In [10]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=100)[0].tolist())
print(generated_chars)

['\n', 'n', 'F', 'o', 'K', ':', 'Q', 'i', 'k', 't', 'o', 'F', '.', '$', 'h', 'x', '-', 'v', '.', '\n', 'e', 'Z', 'U', 'Z', '.', '$', 'y', ':', 'E', 'A', 'F', 'W', 'W', 'H', 'q', 'N', ';', ';', 'z', 'e', '.', ' ', 'r', 'S', '$', 'A', 's', 'e', 'm', '.', ' ', 'q', 'g', 'f', 'r', 'g', 'a', 'r', 'W', 'k', 'M', 'X', 'M', 'L', '\n', 'R', 'Y', 'd', '?', 'Y', 'E', 'i', 'l', '$', 'C', ':', '\n', 'F', 'H', '-', 'p', ' ', 'I', 'Z', 'o', 'y', 'o', 'x', 't', 'o', '3', ';', 'J', 'Z', 'R', 't', '!', 'b', 'r', 'i', 'C']
